In [28]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
import os
import warnings
warnings.filterwarnings('ignore')

In [29]:
# Load the dataset
df = pd.read_csv('sales_data_sample.csv', encoding='latin1')

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2823 entries, 0 to 2822
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ORDERNUMBER       2823 non-null   int64  
 1   QUANTITYORDERED   2823 non-null   int64  
 2   PRICEEACH         2823 non-null   float64
 3   ORDERLINENUMBER   2823 non-null   int64  
 4   SALES             2823 non-null   float64
 5   ORDERDATE         2823 non-null   object 
 6   STATUS            2823 non-null   object 
 7   QTR_ID            2823 non-null   int64  
 8   MONTH_ID          2823 non-null   int64  
 9   YEAR_ID           2823 non-null   int64  
 10  PRODUCTLINE       2823 non-null   object 
 11  MSRP              2823 non-null   int64  
 12  PRODUCTCODE       2823 non-null   object 
 13  CUSTOMERNAME      2823 non-null   object 
 14  PHONE             2823 non-null   object 
 15  ADDRESSLINE1      2823 non-null   object 
 16  ADDRESSLINE2      302 non-null    object 


In [31]:
total_duplicates = df.duplicated().sum()
print(f"Total number of exact duplicate rows: {total_duplicates}")

Total number of exact duplicate rows: 0


In [32]:

df.describe()


,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,QTR_ID,MONTH_ID,YEAR_ID,MSRP
count,2823.000000,2823.000000,2823.000000,2823.000000,2823.000000,2823.000000,2823.000000,2823.00000,2823.000000
mean,10258.725115,35.092809,83.658544,6.466171,3553.889072,2.717676,7.092455,2003.81509,100.715551
std,92.085478,9.741443,20.174277,4.225841,1841.865106,1.203878,3.656633,0.69967,40.187912
min,10100.000000,6.000000,26.880000,1.000000,482.130000,1.000000,1.000000,2003.00000,33.000000
25%,10180.000000,27.000000,68.860000,3.000000,2203.430000,2.000000,4.000000,2003.00000,68.000000
50%,10262.000000,35.000000,95.700000,6.000000,3184.800000,3.000000,8.000000,2004.00000,99.000000
75%,10333.500000,43.000000,100.000000,9.000000,4508.000000,4.000000,11.000000,2004.00000,124.000000
max,10425.000000,97.000000,100.000000,18.000000,14082.800000,4.000000,12.000000,2005.00000,214.000000


In [33]:
df.head(50)

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,...,897 Long Airport Avenue,NaN,NYC,NY,10022,USA,NaN,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,5,2003,...,59 rue de l'Abbaye,NaN,Reims,NaN,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,...,27 rue du Colonel Pierre Avia,NaN,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,...,78934 Hillside Dr.,NaN,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,...,7734 Strong St.,NaN,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium
5,10168,36,96.66,1,3479.76,10/28/2003 0:00,Shipped,4,10,2003,...,9408 Furth Circle,NaN,Burlingame,CA,94217,USA,NaN,Hirano,Juri,Medium
6,10180,29,86.13,9,2497.77,11/11/2003 0:00,Shipped,4,11,2003,...,"184, chausse de Tournai",NaN,Lille,NaN,59000,France,EMEA,Rance,Martine,Small
7,10188,48,100.00,1,5512.32,11/18/2003 0:00,Shipped,4,11,2003,...,"Drammen 121, PR 744 Sentrum",NaN,Bergen,NaN,N 5804,Norway,EMEA,Oeztan,Veysel,Medium
8,10201,22,98.57,2,2168.54,12/1/2003 0:00,Shipped,4,12,2003,...,5557 North Pendale Street,NaN,San Francisco,CA,NaN,USA,NaN,Murphy,Julie,Small
9,10211,41,100.00,14,4708.44,1/15/2004 0:00,Shipped,1,1,2004,...,"25, rue Lauriston",NaN,Paris,NaN,75016,France,EMEA,Perrier,Dominique,Medium


In [34]:
# 1. Identify and handle missing values
missing_summary = df.isnull().sum()
print('Missing values per column:')
print(missing_summary)

Missing values per column:
ORDERNUMBER            0
QUANTITYORDERED        0
PRICEEACH              0
ORDERLINENUMBER        0
SALES                  0
ORDERDATE              0
STATUS                 0
QTR_ID                 0
MONTH_ID               0
YEAR_ID                0
PRODUCTLINE            0
MSRP                   0
PRODUCTCODE            0
CUSTOMERNAME           0
PHONE                  0
ADDRESSLINE1           0
ADDRESSLINE2        2521
CITY                   0
STATE               1486
POSTALCODE            76
COUNTRY                0
TERRITORY           1074
CONTACTLASTNAME        0
CONTACTFIRSTNAME       0
DEALSIZE               0
dtype: int64


In [35]:
# 1b. Fill or drop missing values (example: fill with mode or drop)
df = df.fillna(df.mode().iloc[0])  # or use df.dropna()

In [36]:
# 2. Remove duplicate rows
df = df.drop_duplicates()

In [37]:
# 3. Standardize text values (example: gender, country)
if 'gender' in df.columns:
    df['gender'] = df['gender'].str.strip().str.lower().replace({'f':'female','m':'male'})
if 'country' in df.columns:
    df['country'] = df['country'].str.strip().str.title()

In [38]:
# 4. Convert date formats to consistent type (example: order_date column)
date_cols = [col for col in df.columns if 'date' in col.lower()]
for col in date_cols:
    try:
        df[col] = pd.to_datetime(df[col], errors='coerce', dayfirst=True)
    except Exception as e:
        print(f'Could not convert {col}:', e)

In [39]:
# 5. Rename column headers to be clean and uniform
df.columns = [re.sub(r'[^a-z0-9_]', '', col.lower().replace(' ', '_')) for col in df.columns]

In [41]:
# 7. Save cleaned data
output_path = 'cleaned_sales_data_sample.csv'
df.to_csv(output_path, index=False)
print(f'Cleaned data saved to {output_path}')

Cleaned data saved to cleaned_sales_data_sample.csv


In [43]:
# Get missing value and duplicate info for README
missing_summary = df.isnull().sum()
total_missing = missing_summary.sum()
total_duplicates = df.duplicated().sum()
print('Total missing values:', total_missing)
print('Missing values per column:')
print(missing_summary)
print('Total duplicate rows:', total_duplicates)

Total missing values: 0
Missing values per column:
ordernumber         0
quantityordered     0
priceeach           0
orderlinenumber     0
sales               0
orderdate           0
status              0
qtr_id              0
month_id            0
year_id             0
productline         0
msrp                0
productcode         0
customername        0
phone               0
addressline1        0
addressline2        0
city                0
state               0
postalcode          0
country             0
territory           0
contactlastname     0
contactfirstname    0
dealsize            0
dtype: int64
Total duplicate rows: 0


In [ ]:
# Capture statistics BEFORE cleaning
df = pd.read_csv('sales_data_sample.csv', encoding='latin1')

print('=== DATA BEFORE CLEANING ===')
print(f'Total rows: {len(df)}')
print(f'Total columns: {len(df.columns)}')
total_missing_before = df.isnull().sum().sum()
total_duplicates_before = df.duplicated().sum()
print(f'Total missing values: {total_missing_before}')
print(f'Total duplicate rows: {total_duplicates_before}')
print('\nMissing values per column:')
print(df.isnull().sum())

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3882640606.py, line 10)